# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
import xarray as xr
import h5py
from tqdm.auto import tqdm

import numpy as np
import matplotlib.pyplot as plt

# Misc
from scipy.ndimage import gaussian_filter1d

# Open nexus files
from nexusformat.nexus import *

# Self-written libraries
sys.path.append(join(os.getcwd(),"library"))
import support_functions as sup
from interactive import cimshow

# Gifs
import imageio

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets
%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018955
USER = getuser()

In [ ]:
BASEFOLDER = "/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2403_tomo"


# Load any kind of data from collection
def load_pre_scan_snapshot(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id
    
    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"]["pre_scan_snapshot"][key][()])
        print("Loaded: %s" % (fname))
    return np.squeeze(data)

# Load any kind of data from measurements
def load_data(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id

    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"][key][()])
        print("Loaded key %s of %s" % (key, entry))
    return np.squeeze(data)

In [ ]:
# Not implemented yet
def norm_hyst_data(ydata, norm):
    """
    Normalize hysteresis yaxis to +/- 1
    """
    ynorm = []
    for i in range(len(ydata)):
        tmp = ydata[i] / norm[i]
        tmp = tmp - np.mean(tmp[-5:])
        tmp = tmp / np.mean(tmp[:5])
        tmp = 2 * (tmp - 0.5)
        ynorm.append(tmp)

    return ynorm

In [ ]:
from scipy.optimize import curve_fit
def gaussian(x, a, x0, sigma, c):
    return a * np.exp(-((x - x0) ** 2) / (2 * sigma**2)) + c

# Experimental Details

In [ ]:
# Dict with basic experimental parameter
experimental_setup = {}

# General saving folder
folder_general = sup.create_folder(join(BASEFOLDER, "processed"))
print("Output Folder: %s" % folder_general)


# Single Diode scans

In [ ]:
# Add scan numbers to combine in single plot
scan_ids = [249,250]

# keys from nxs files (if you are not sure, which you need, open nexus files from beamtime folder and navigate through keys `2310_P04_FGT/raw/FGT`)
data_key_x = "energy" # "energy", "sx", "sy"
data_key_y = "maxibeckhoff1adc1_rmean"

# Normalization of diode signal with I0, e.g., with petra current or gold mesh (needs to be inserted during scan)
normalization = False
data_key_norm = "maxibeckhoff1adc2_rmean"  # "petra_curr"


# Load energy/wavelength and add to experimental setup
try:
    experimental_setup["energy"] = np.array(load_energy(scan_ids[0]))
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
except:
    pass

# Load data
xdata, ydata, norm = [], [], []
for scan_id in tqdm(scan_ids):
    xdata.append(load_data(scan_id, data_key_x))
    ydata.append(load_data(scan_id, data_key_y))

    if normalization is True:
        norm.append(load_data(scan_id, data_key_norm))
    else:
        norm.append(np.ones(ydata[-1].shape))

In [ ]:
# Format title and output file name
if len(scan_ids) > 1:
    title = "ScanIds: %04d-%04d" % (scan_ids[0], scan_ids[-1])
    fname = join(
        folder_general,
        "ScanId_%04d-%04d_Diode_%s.png" % (scan_ids[0], scan_ids[-1], USER),
    )
else:
    title = "ScanId: %04d" % (scan_ids[0])
    fname = join(folder_general, "ScanId_%04d_Diode_%s.png" % (scan_ids[0], USER))


# Plot all scans together
fig, ax = plt.subplots()

# Iterate over scans
for i, _ in enumerate(scan_ids):
    ax.plot(
        xdata[i],
        ydata[i] / norm[i],
        "o-",
        markersize=2,
        label="Id: %d" % scan_ids[i],
    )

# Plotting options
ax.set_title(title)
ax.set_xlabel(data_key_x)
# ax.set_xlim([-0.01, 0.01])
# ax.set_ylim([-0.015, 0.015])
ax.grid()
ax.legend()

# Different label with and without normalization
if normalization is True:
    ax.set_ylabel(data_key_y + " (normalized by %s)" % data_key_norm, fontsize=7)
else:
    ax.set_ylabel(data_key_y + " (a.u.)")

# Save plot
print("Saving: %s" % fname)
plt.savefig(fname, dpi=300)

## Fitting of scans with a gaussian

In [ ]:
# Add scan numbers to combine in single plot
scan_ids = [173]

# keys from nxs files (if you are not sure, which you need, open nexus files from beamtime folder and navigate through keys `2310_P04_FGT/raw/FGT`)
data_key_x = "sy"
data_key_y = "maxibeckhoff1adc1_rmean"

# Normalization of diode signal with I0, e.g., with petra current or gold mesh (needs to be inserted during scan)
normalization = False
data_key_norm = "adc_beck_femto_goldmesh_mean"  # "petra_curr"


# Load energy/wavelength and add to experimental setup
try:
    experimental_setup["energy"] = np.array(load_energy(scan_ids[0]))
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
except:
    pass

# Load data
xdata, ydata, norm = [], [], []
for scan_id in tqdm(scan_ids):
    xdata.append(load_data(scan_id, data_key_x))
    ydata.append(load_data(scan_id, data_key_y))

    if normalization is True:
        norm.append(load_data(scan_id, data_key_norm))
    else:
        norm.append(np.ones(ydata[-1].shape))

In [ ]:
# Format title and output file name
if len(scan_ids) > 1:
    title = "ScanIds: %04d-%04d" % (scan_ids[0], scan_ids[-1])
    fname = join(
        folder_general,
        "ScanId_%04d-%04d_Fit_%s.png" % (scan_ids[0], scan_ids[-1], USER),
    )
else:
    title = "ScanId: %04d" % (scan_ids[0])
    fname = join(folder_general, "ScanId_%04d_Fit_%s.png" % (scan_ids[0], USER))


# Plot all scans together
fig, ax = plt.subplots()

fit = True

# Iterate over scans
for i, scan_id in enumerate(scan_ids):
    # Plot diode
    y = ydata[i] / norm[i]
    p = ax.plot(
        xdata[i],
        y,
        "o",
        markersize=2,
        label="Id: %d" % scan_ids[i],
    )
    
    # Initial fitting Parameter
    if fit is True:
        A0 = np.max(ydata[i] / norm[i])
        x0 = xdata[i][np.argmax(ydata[i])]
        sigma0 = 0.01
        c0 = 0  # np.min(ydata[i])
        popt, pcov = curve_fit(
            gaussian, xdata[i], y, p0=(A0, x0, sigma0, c0)
        )
        # popt returns the best fit values for parameters of the given model (func)
        print("Scan %d: A=%02f, x=%02f, sigma=%02f, c=%02f"%(scan_id,popt[0],popt[1],popt[2],popt[3]))
        color = p[0].get_color()
        # plot fit
        x = np.linspace(xdata[i][0], xdata[i][-1], 100)
        ym = gaussian(x, popt[0], popt[1], popt[2], popt[3])
        ax.plot(
            x, ym, color=color, label="FWHM = %.3f" % (2 * np.sqrt(2 * np.log(2)) * popt[2])
        )

        # Add vline line
        ax.axvline(popt[1], 0, 10, color=color)

# Plotting options
ax.set_title(title)
ax.set_xlabel(data_key_x)
# ax.set_xlim([-0.01, 0.01])
# ax.set_ylim([-0.015, 0.015])
ax.grid()
ax.legend()

# Different label with and without normalization
if normalization is True:
    ax.set_ylabel(data_key_y + " (normalized by %s)" % data_key_norm, fontsize=7)
else:
    ax.set_ylabel(data_key_y + " (a.u.)")

# Save plot
print("Saving: %s" % fname)
plt.savefig(fname)

# Diode Trace Hysteresis

In [ ]:
def ramp_tan(npts: int, s: float, full=False):
    """Calculate truncated tangent ramp function.

    Parameters
    ==========
    npts: int
        Number of points to calculate
    s: float
        shape parameter (0<s≤1) varies curve from linear (s→0) to tan-shaped
    full: bool
        If True, return both arms of the loop by mirroring. This effectively
        doubles the number of points.

    Returns
    =======
    ramp: np.ndarray
        normalized ramp values
    """
    # if (s <= 0) or (s > 1):
    #     raise ValueError("shape parameter out of range 0<s≤1")
    x = s * np.linspace(-np.pi / 2, np.pi / 2, npts)
    y = np.tan(x) / np.tan(np.pi * s / 2)
    if full:
        y = np.append(y, y[::-1][1:])
    return y

In [ ]:
def normalize(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())

In [ ]:
# Choose scan id
scan_id = 2511

# Load trace data
fields = [
    f"adc_beck_femto_diodemax_trace",
    "adc_beck_femto_goldmesh_trace",
    "adc_beck_mtesla_trace",
]
adcs = [load_data(scan_id, f) for f in fields]

In [ ]:
# Create xarray dataset
ds = xr.Dataset()
ds["fields"] = ("index", adcs[2][:, 10:-10].flatten())  # magnetic field
ds["diode"] = ("index", adcs[0][:, 10:-10].flatten())  # diode current
ds["mesh"] = ("index", adcs[1][:, 10:-10].flatten())  # mesh current for normalization
ds["rising"] = (
    "index",
    gaussian_filter1d(np.gradient(ds["fields"]), 20) > 0,
)  # which hysteresis branch

In [ ]:
bins = np.max(ds.fields.values) * ramp_tan(100, 0.5, False)

ds_sel = ds
ds_sel["rising"] = ds_sel.rising.astype(bool)
up = ds_sel.where(ds_sel.rising).groupby_bins("fields", bins).mean()
down = ds_sel.where(~ds_sel.rising).groupby_bins("fields", bins).mean()

In [ ]:
fig, ax = plt.subplots()
ax.plot(1e2 * up.fields, 2 * normalize(up.diode / up.mesh) - 1, "o", ms=3)
ax.plot(1e2 * down.fields, 2 * normalize(down.diode / down.mesh) - 1, "o", ms=3)
ax.set_xlabel("field (mT)")
ax.set_ylabel("diode / mesh (arb. units)")
ax.set_title("scan %d: field sweep @ 0.2Hz" % (scan_id))
ax.grid()

fname = join(folder_general, "ScanId_%04d_Diode_Trace_%s.png" % (scan_id, USER))
print("Saving: %s" % fname)
plt.savefig(fname, dpi=300)